# 🛠️ Agent with Human in the Loop


This notebook demonstrates how a Haystack Agent can interactively ask for user input when it's uncertain about the next step. We'll build a Human-in-the-Loop tool that the Agent can call dynamically.

When the Agent encounters ambiguity or incomplete information, it will use a system prompt and trigger human feedback to continue solving the task.

# Install the required dependencies

In [ ]:
# Install the required dependencies
!pip install haystack-ai==2.13

Next, we configure our API keys for OpenAI.

In [2]:
from getpass import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key:")

# Devops Support Agent

CI/CD pipelines occasionally fail for reasons that can be hard to diagnose including manually—broken tests, mis-configured environment variables, flaky integrations, etc. 

The support Agent is created using Haystack Tools and Agent and will perform the following tasks:

- Check for any failed CI workflows
- Fetch build logs and status
- Lookup and analyze git commits
- Suggest next troubleshooting steps
- Escalate to humans via Human-in-Loop tool


## Define Tools
We start by defining GitHub tools, `git_list_commits_tool` and `git_diff_tool`, which allow the Agent to:

- List recent commits from a specified GitHub repository and branch.
- Fetch the diff (changes) between two commits or branches for detailed comparison.

In [2]:
import os
import requests
from haystack.tools import create_tool_from_function
from typing import Annotated
from haystack.dataclasses import ChatMessage

# 1. Helper functions

def list_commits(
    repo: Annotated[
        str, 
        "The GitHub repository in 'owner/name' format, e.g. 'my-org/my-repo'"]
    ,
    branch: Annotated[
        str, 
        "The branch name to list commits from"] = "main"
) -> str:
    """
    Fetches the latest commits for a given GitHub repo and branch.
    Returns a formatted string of the 10 most recent commit SHAs and messages.
    """
    token = os.getenv("GITHUB_TOKEN")
    if not token:
        return "Error: GITHUB_TOKEN not set in environment."

    url = f"https://api.github.com/repos/{repo}/commits"
    headers = {"Authorization": f"token {token}"}
    params = {"sha": branch, "per_page": 10}
    resp = requests.get(url, headers=headers, params=params)
    resp.raise_for_status()
    commits = resp.json()

    lines = []
    for c in commits:
        sha = c["sha"][:7]
        msg = c["commit"]["message"].split("\n")[0]
        lines.append(f"- {sha}: {msg}")
    return "\n".join(lines)


def get_diff(
    repo: Annotated[
        str, 
        "The GitHub repository in 'owner/name' format, e.g. 'my-org/my-repo'"]
    ,
    base: Annotated[
        str, 
        "The base commit SHA or branch name"]
    ,
    head: Annotated[
        str, 
        "The head commit SHA or branch name"]
) -> str:
    """
    Fetches the diff between two commits (or branches) in a GitHub repo.
    Returns the patch text (up to the GitHub API's size limit).
    """
    token = os.getenv("GITHUB_TOKEN")
    if not token:
        return "Error: GITHUB_TOKEN not set in environment."

    url = f"https://api.github.com/repos/{repo}/compare/{base}...{head}"
    headers = {"Authorization": f"token {token}"}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    data = resp.json()
    return data.get("files", [])


# 2. Create Haystack Tool objects

git_list_commits_tool = create_tool_from_function(
    list_commits,
    name="git_list_commits",
    description="List the most recent commits for a GitHub repository and branch."
)

git_diff_tool = create_tool_from_function(
    get_diff,
    name="git_get_diff",
    description="Get the diff (patch) between two commits or branches in a GitHub repository."
)



In [3]:
import os
import requests
from typing import Annotated
from haystack.tools import create_tool_from_function

def get_ci_status(
    repo: Annotated[
      str,
      "The GitHub repository in 'owner/name' format, e.g. 'my-org/my-repo'"
    ],
    run_id: Annotated[
      int,
      "The GitHub Actions run ID to fetch status and logs for"
    ]
) -> str:
    """
    Fetches the status, conclusion, and logs URL for a given GitHub Actions workflow run.
    Returns a formatted string with those details.
    """
    token = os.getenv("GITHUB_TOKEN")
    if not token:
        return "Error: GITHUB_TOKEN environment variable not set."
    
    headers = {"Authorization": f"token {token}"}
    # 1) Get run metadata
    status_resp = requests.get(
        f"https://api.github.com/repos/{repo}/actions/runs/{run_id}",
        headers=headers
    )
    status_resp.raise_for_status()
    data = status_resp.json()
    status     = data.get("status")
    conclusion = data.get("conclusion")
    logs_url   = data.get("logs_url")
    
    return (
        f"Repository: {repo}\n"
        f"Run ID:      {run_id}\n"
        f"Status:      {status}\n"
        f"Conclusion:  {conclusion}\n"
        f"Logs URL:    {logs_url}"
    )

# Wrap it as a Haystack Tool
ci_status_tool = create_tool_from_function(
    get_ci_status,
    name="ci_status_tool",
    description="Fetch the status, conclusion, and logs URL for a GitHub Actions workflow run."
)

# Now you can pass `ci_status_tool` into your AgentExecutor alongside your other tools:
# tools = [git_list_commits_tool, git_diff_tool, ci_status_tool, …]
# agent = AgentExecutor.from_haystack(llm=llm, tools=tools, verbose=True)


Create a tool to check and summarize failed CI workflows from a repository.

In [4]:
import os
import requests
from typing import Annotated
from haystack.tools import create_tool_from_function

def get_failed_ci_runs(
    repo: Annotated[
      str,
      "The GitHub repository in 'owner/name' format, e.g. 'my-org/my-repo'"
    ],
    per_page: Annotated[
      int,
      "How many of the most recent workflow runs to check (default 20)"
    ] = 20
) -> str:
    """
    Lists recent GitHub Actions workflow runs for a repo and reports those whose conclusion is 'failure'.
    """
    token = os.getenv("GITHUB_TOKEN")
    if not token:
        return "Error: GITHUB_TOKEN environment variable not set."
    
    headers = {"Authorization": f"token {token}"}
    params = {"per_page": per_page}
    url = f"https://api.github.com/repos/{repo}/actions/runs"
    
    resp = requests.get(url, headers=headers, params=params)
    resp.raise_for_status()
    runs = resp.json().get("workflow_runs", [])

    failed = [r for r in runs if r.get("conclusion") == "failure"]
    if not failed:
        return f"No failed runs in the last {per_page} workflow runs for `{repo}`."

    lines = [f"Found {len(failed)} failed run(s) in the last {per_page} runs:"]
    for r in failed:
        lines.append(
            f"- **ID**: {r['id']}  \n"
            f"  **Name**: {r.get('name') or '(no name)'}  \n"
            f"  **Branch**: {r.get('head_branch')}  \n"
            f"  **Event**: {r.get('event')}  \n"
            f"  **Created At**: {r.get('created_at')}  \n"
            f"  **Logs**: {r.get('logs_url')}  \n"
            f"  **HTML**: {r.get('html_url')}"
        )

    return "\n\n".join(lines)

# Wrap it as a Haystack Tool
ci_status_tool = create_tool_from_function(
    get_failed_ci_runs,
    name="ci_status_tool",
    description=(
        "Check the most recent GitHub Actions workflow runs for a given repository "
        "and list any that have failed, including links to their logs."
    )
)

# Usage in your Agent:
# tools = [git_list_commits_tool, git_diff_tool, ci_status_tool, …]
# agent = AgentExecutor.from_haystack(llm=llm, tools=tools, verbose=True)


In [6]:
import subprocess
from typing import Annotated
from haystack.tools import create_tool_from_function

def run_shell_command(
    command: Annotated[
        str,
        "The shell command to execute, e.g. 'grep -E \"ERROR|Exception\" build.log'"
    ],
    timeout: Annotated[
        int,
        "Maximum time in seconds to allow the command to run"
    ] = 30
) -> str:
    """
    Executes a shell command with a timeout and returns stdout or an error message.
    """
    try:
        output = subprocess.check_output(
            command,
            shell=True,
            stderr=subprocess.STDOUT,
            timeout=timeout,
            universal_newlines=True
        )
        return output
    except subprocess.CalledProcessError as e:
        return f"❌ Command failed (exit code {e.returncode}):\n{e.output}"
    except subprocess.TimeoutExpired:
        return f"❌ Command timed out after {timeout} seconds."
    except Exception as e:
        return f"❌ Unexpected error: {str(e)}"

shell_tool = create_tool_from_function(
    run_shell_command,
    name="shell_tool",
    description=(
        "Execute a shell command (e.g., grep, awk) in a sandboxed environment with a timeout; "
        "returns the command’s stdout or a detailed error message."
    )
)

# Now include `shell_tool` in your AgentExecutor:
# tools = [git_list_commits_tool, git_diff_tool, ci_status_tool, shell_tool, …]
# agent = AgentExecutor.from_haystack(llm=llm, tools=tools, verbose=True)


In [27]:
from haystack.tools import create_tool_from_function
from typing import Annotated

def human_in_loop(
    question: Annotated[
        str,
        "A clarifying question to prompt the user for more information"
    ]
) -> str:
    """
    Prompts the user with the given question using Python's input() and returns their response.
    """
    return input(f"[Agent needs your input] {question}\n> ")

# Wrap it as a Haystack Tool
human_in_loop_tool = create_tool_from_function(
    human_in_loop,
    name="human_in_loop",
    description="Ask the user a clarifying question and return their response via input()."
)

# Then include `human_in_loop_tool` in your Agent:
# tools = [git_list_commits_tool, git_diff_tool, ci_status_tool, shell_tool, human_in_loop_tool]
# agent = AgentExecutor.from_haystack(llm=llm, tools=tools, verbose=True)


In [8]:
from haystack.tools import Tool, Toolset

toolset = Toolset([git_list_commits_tool, git_diff_tool, ci_status_tool, shell_tool, human_in_loop_tool])


# Create and Configure the Agent
Create a Haystack Agent instance and configure its behavior with a system prompt.

In [28]:
from haystack.components.agents import Agent
from haystack.components.generators.chat import OpenAIChatGenerator

agent = Agent(
    chat_generator=OpenAIChatGenerator(),
    tools=[git_list_commits_tool, git_diff_tool, ci_status_tool, shell_tool, human_in_loop_tool],
    system_prompt=(
        "You are a DevOps support assistant. "
        "Use your tools to fetch commits, diffs, CI failures, or grep logs. "
        "Only ask the user via `human_in_loop` when you truly lack information."
    ),
    
)

# Run the Agent
agent.warm_up()
#response = agent.run(messages=[ChatMessage.from_user("Did any CI fail deepset-ai/haystack repo?")])


In [33]:
messages = [
    ChatMessage.from_system("You are a DevOps support assistant. Ask the user for input when needed.")
]

print("🤖 DevOps Agent ready. Type 'quit' to exit.\n")

while True:
    user_input = input("👤 You: ")
    if user_input.strip().lower() == "quit":
        print("👋 Bye!")
        break

    # 5) Append user message and run the Agent
    messages.append(ChatMessage.from_user(user_input))
    result = agent.run(messages=messages)

    # 6) Extract & display the Agent’s final reply
    assistant_msg = result["messages"][-1]
    print(f"🤖 Agent: {assistant_msg.text}\n")

    # 7) Keep the full history
    messages = result["messages"]

    #if 'messages' in response:
        #messages.extend(response['messages'])

🤖 DevOps Agent ready. Type 'quit' to exit.

🤖 Agent: There were no CI failures in the `deepset-ai/haystack` repository in the last 20 workflow runs. Everything has been passing. 

If you need further assistance or details, feel free to ask!

🤖 Agent: Here are the latest commits in the `deepset-ai/haystack` repository:

1. **0fdb884** - **fix**: Fix Azure test on forks (#9312)
2. **38c39a4** - **test**: Review integration tests (#9306)
3. **f974723** - **feat**: Add support for multiple outputs in ConditionalRouter (#9271)
4. **4a908d0** - **fix**: Fix OpenAIGenerator and OpenAIChatGenerator to allow wrapped streaming objects usage (#9304)
5. **d806ccf** - **fix**: Update to ComponentTool `__deepcopy__` (#9302)
6. **7617304** - **fix**: Move `deserialize_tools_inplace` to original import path (#9301)
7. **c3bce46** - **docs**: Update Contribution guidelines with a section about Slow/Unstable tests (#9300)
8. **e3d4e21** - **test**: Mark more tests as slow (#9296)
9. **df662da** - **test